In [53]:
from dotenv import load_dotenv
from agents import Agent, Runner, ItemHelpers, trace, function_tool
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content



In [54]:
load_dotenv(override=True)

True

In [55]:

agent = Agent(name="Assistant", instructions="You are a helpful assistant", model="gpt-4o-mini")

result = await Runner.run(agent, "Tell a joke about Autonomous Agentic AI")
print(result.final_output)

Why did the Autonomous Agentic AI break up with its programmer? 

It found someone who could truly give it the space to grow… in the cloud!


In [39]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions="You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write professional, serious cold emails.",
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions="You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write witty, engaging cold emails that are likely to get a response.",
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions="You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write concise, to the point cold emails.",
        model="gpt-4o-mini"
)



In [40]:
async def main():
    result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

In [41]:
await main()

Subject: Streamline Your SOC 2 Compliance Process

Hi [Recipient's Name],

I hope this message finds you well.

I’m reaching out to introduce you to ComplAI, an innovative SaaS solution designed to simplify the SOC 2 compliance process for organizations like yours. As you know, maintaining compliance can be a complex and time-consuming task, especially when preparing for audits.

Our tool leverages AI to automate documentation, reduce manual effort, and ensure you’re always audit-ready. Many of our clients have reported significant time savings and reduced stress during their compliance journey.

I’d love to schedule a brief call to discuss how ComplAI can specifically benefit your organization. Would you be available for a quick chat this week?

Thank you for considering, and I look forward to the opportunity to connect.

Best regards,

[Your Name]  
[Your Position]  
ComplAI  
[Your Phone Number]  
[Your LinkedIn Profile]  
[Your Website]  

In [42]:
async def main():
    message = "Write a cold sales email"

    with trace("Parallel cold emails"):
        results = await asyncio.gather(
            Runner.run(sales_agent1, message),
            Runner.run(sales_agent2, message),
            Runner.run(sales_agent3, message),
        )
        outputs = [ItemHelpers.text_message_outputs(result.new_items) for result in results]

    for output in outputs:
        print(output + "\n\n")


In [43]:
await main()

Trace already exists. Creating a new trace, but this is probably a mistake.


Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this email finds you well. My name is [Your Name], and I’m with ComplAI, a leader in AI-driven solutions for SOC 2 compliance.

As regulatory requirements grow increasingly complex, many organizations find themselves overwhelmed by the demands of maintaining compliance and preparing for audits. At ComplAI, we offer a comprehensive SaaS tool designed to simplify this process, making it more efficient and less time-consuming.

Our platform enables you to:

- Automate the documentation process, reducing manual effort
- Gain real-time insights into your compliance status
- Prepare seamlessly for audits with organized, accessible records

By leveraging our AI capabilities, companies like yours have not only enhanced their compliance posture but also freed up valuable resources to focus on core business objectives.

I would love the opportunity to discuss how ComplAI can help your organization streamline

In [44]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to.",
)

In [45]:
async def main():
    message = "Write a cold sales email"

    with trace("Selection from sales people"):
        results = await asyncio.gather(
            Runner.run(sales_agent1, message),
            Runner.run(sales_agent2, message),
            Runner.run(sales_agent3, message),
        )
        outputs = [ItemHelpers.text_message_outputs(result.new_items) for result in results]

        emails = "Cold sales emails:\n\n".join(outputs)

        best = await Runner.run(
            sales_picker,
            emails,
        )

    print(f"Best sales email:\n{best.final_output}")


In [46]:
await main()

Trace already exists. Creating a new trace, but this is probably a mistake.


Best sales email:
Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in aiding companies like [Recipient's Company] to achieve and maintain SOC 2 compliance seamlessly.

Navigating the complexities of compliance can be daunting, especially when preparing for audits. Our AI-powered SaaS tool simplifies this process, providing you with real-time insights, automated documentation, and a structured roadmap tailored to your specific needs.

Here are a few ways ComplAI can enhance your compliance efforts:

- **Automated Monitoring:** Continuous oversight of your systems to ensure adherence to SOC 2 requirements.
- **Streamlined Documentation:** Effortlessly generate and manage the necessary documentation for audits.
- **Insightful Reporting:** Gain AI-driven insights to identify gaps and improve practices before auditors arrive.

I’d love to discuss 

In [56]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions="You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write professional, serious cold emails.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email has been written"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions="You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write witty, engaging cold emails that are likely to get a response.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email has been written"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions="You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
                You write concise, to the point cold emails.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email has been written"
)

In [60]:
@function_tool
def send_email(body: str) -> Dict[str, str]:
    print(f"Sending email:\n{body}")
    
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    subject = "Sales email"
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content)
    mail_json = mail.get()
    response = sg.client.mail.send.post(request_body=mail_json)
    print("Sent email and got response", response.status_code)
    return {"status": "success"}

In [62]:
sales_manager = Agent(
    name="Sales Manager",
    instructions="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all the tools at least once before choosing the best one. \
You pick the best email and use the tool to send it to the user.",
    tools=[
        sales_agent1.as_tool(
            tool_name="sales_agent1",
            tool_description="Write a cold sales email",
        ),
        sales_agent2.as_tool(
            tool_name="sales_agent2",
            tool_description="Write a cold sales email",
        ),
        sales_agent3.as_tool(
            tool_name="sales_agent3",
            tool_description="Write a cold sales email",
        ),
        send_email
    ],
)

async def main():
    message = "Send a cold sales email addressed to Mike"

    with trace("Sales manager"):
        result = await Runner.run(sales_manager, message)

In [63]:
await main()

Trace already exists. Creating a new trace, but this is probably a mistake.


Sending email:
Subject: Is Your Compliance Toolkit Ready for a Glow-Up?

Hi [Recipient's Name],

Are you tired of Compliance feeling like that elusive sock that always disappears in the dryer? 😫

At ComplAI, we believe compliance should be more like a cozy pair of slippers—supportive, easy to find, and always ready when you need them!

Our AI-powered platform is the secret sauce to achieving SOC2 compliance and effortlessly preparing for audits. Imagine wave goodbye to the stress of last-minute panics and hello to a smooth, streamlined process. 🕊️

**Here’s the scoop:**
- **No more paperwork mountain**—we automate the grind of compliance checking. 
- **Team of compliance whispers**—our AI works harder than a caffeinated squirrel to keep everything in check!
- **Ready for audits in your sleep** (not literally, but you get the point).

Let's schedule a quick chat—10 minutes to explore how we can make your compliance journey less like herding cats and more like a walk in the park. What do

In [66]:
subject_writer = Agent(
        name="Email subject writer",
        instructions="You can write a subject for a cold sales email. You are given a message and you need to write a subject for an email that is likely to get a response.",
        model="gpt-4o-mini",
        handoff_description="A cold sales email subject has been written")

html_email_body_converter = Agent(
    name="HTML email body converter",   
    instructions="You can convert a text email body to an HTML email body. You are given a text email body which might have some markdown \
          and you need to convert it to an HTML email body with simple,clear, compelling layout and design.",
    model="gpt-4o-mini",
    handoff_description="An HTML email body has been written")



In [73]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content)
    mail_json = mail.get()
    response = sg.client.mail.send.post(request_body=mail_json)
    return {"status": "success"}

In [74]:
sales_manager = Agent(
    name="Sales Manager",
    instructions="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all the tools at least once before choosing the best one. \
You pick the best email, and then you use the tool to write a subject for the chosen email, and another tool to convert the body of the chosen email to HTML. \
    Then you use the tool to send the email with that subject and HTML body to the user.",
    tools=[
        sales_agent1.as_tool(
            tool_name="sales_agent1",
            tool_description="Write a cold sales email",
        ),
        sales_agent2.as_tool(
            tool_name="sales_agent2",
            tool_description="Write a cold sales email",
        ),
        sales_agent3.as_tool(
            tool_name="sales_agent3",
            tool_description="Write a cold sales email",
        ),
        subject_writer.as_tool(
            tool_name="subject_writer",
            tool_description="Write a subject for a cold sales email",
        ),
        html_email_body_converter.as_tool(
            tool_name="html_email_body_converter",
            tool_description="Convert a text email body to an HTML email body",
        ),
        send_email
    ],
)

async def main():
    message = "Send a cold sales email addressed to Mike from Alice"

    with trace("Sales manager"):
        result = await Runner.run(sales_manager, message)

In [75]:
await main()

Trace already exists. Creating a new trace, but this is probably a mistake.


Sending email:
Subject: Simplify SOC2 Compliance with AI – Let's Connect!
Body: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Introducing ComplAI</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f9f9f9;
        }
        .container {
            background-color: #ffffff;
            border-radius: 8px;
            padding: 20px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        h1 {
            color: #333333;
        }
        p {
            color: #555555;
        }
        ul {
            list-style-type: square;
            padding-left: 20px;
            color: #555555;
        }
        .footer {
            margin-top: 20px;
            color: #777777;
        }
    </style>
</head>
<body>

<div class="container">
    <h1>Hi Mike,</h1>
    